# Testing BGE-M3 SageMaker Endpoint

This notebook demonstrates how to test the BGE-M3 embedding model deployed on Amazon SageMaker.

In [ ]:
import boto3
import json
import numpy as np

# Initialize SageMaker Runtime client
smr_client = boto3.client("sagemaker-runtime")

# Replace with your endpoint name
endpoint_name = "YOUR_ENDPOINT_NAME_HERE"

def get_vector_by_sm_endpoint(questions, smr_client, endpoint_name):
    """Get embeddings from SageMaker endpoint"""
    response_model = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                'return_sparse': True,
                'return_colbert_vecs': True,
            }
        ),
        ContentType="application/json",
    )
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    return json_obj

def cos_sim(vector1, vector2):
    """Calculate cosine similarity between two vectors"""
    dot_product = np.dot(vector1, vector2)
    norm_v1 = np.linalg.norm(vector1)
    norm_v2 = np.linalg.norm(vector2)
    cos_sim = dot_product / (norm_v1 * norm_v2)
    return cos_sim

## Test Dense Embeddings

In [ ]:
# Test dense embeddings
text1 = "How cute your dog is!"
text2 = "Your dog is so cute."
text3 = "The mitochondria is the powerhouse of the cell."

# Get dense embeddings and calculate similarity
emb1, emb2, emb3 = get_vector_by_sm_endpoint([text1, text2, text3], smr_client, endpoint_name)['dense_embeddings']
print(f"Similarity between text1 and text2: {cos_sim(emb1, emb2)}")
print(f"Similarity between text1 and text3: {cos_sim(emb1, emb3)}")

## Test Sparse Embeddings

In [ ]:
# Test sparse embeddings
sparse_embeddings = get_vector_by_sm_endpoint([text1, text2, text3], smr_client, endpoint_name)['sparse_embeddings']
sparse_embeddings

## Test ColBERT Vectors

In [ ]:
# Test ColBERT vectors
cvec1, cvec2, cvec3 = get_vector_by_sm_endpoint([text1, text2, text3], smr_client, endpoint_name)['colbert_vectors']
print(f"ColBERT vector dimensions: {len(cvec1[0])}, {len(cvec2[0])}, {len(cvec3[0])}")